In [5]:

import pandas as pd 
import numpy as np
from datetime import date, datetime, timedelta
import lxml.html
import urllib3
import json
import requests 
import time
import random

url = 'https://servicios.cmfchile.cl/simuladorhipotecario/aplicacion?indice=101.2.3 \
&maxuf=20000&minuf=100&maxpeso=652419400&minpeso=3262097&paso=2&template=entidades&tipomoneda=1 \
&monto=3000&tipocredito=3&tipotasa=1&plazo=20&inst=OK& \
marcados=1&marcados=9&marcados=12&marcados=14&marcados=16&marcados=28&marcados=37&marcados=39&marcados=51&marcados=672'

session=requests.Session()
session.verify=False

r = session.get( url.replace(' ',''), headers={
'Upgrade-Insecure-Requests': '1',
'Connection': 'keep-alive',
'Content-Type': 'application/x-www-form-urlencoded', \
'Referer': 'https://servicios.cmfchile.cl/simuladorhipotecario/aplicacion?indice=101.2.1', \
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'} )
root = lxml.html.fromstring(r.text)

u:\Python\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'servicios.cmfchile.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [18]:
#test cuantos bancos en el listado 
nbancos=len(root.xpath("////*/div[@id = 'simuladorCreditoHipotecario']/table/tbody/tr"))
for i in range(1,11):
  banco=root.xpath("////*/div[@id = 'simuladorCreditoHipotecario']/table/tbody/tr["+ str(i) + "]/td")[0].text
  tipo=root.xpath("////*/div[@id = 'simuladorCreditoHipotecario']/table/tbody/tr["+ str(i) + "]/td")[1].text
  dividendo=root.xpath("////*/div[@id = 'simuladorCreditoHipotecario']/table/tbody/tr["+ str(i) + "]/td")[2].text
  moneda=root.xpath("////*/div[@id = 'simuladorCreditoHipotecario']/table/tbody/tr["+ str(i) + "]/td")[3].text
  tasa=root.xpath("////*/div[@id = 'simuladorCreditoHipotecario']/table/tbody/tr["+ str(i) + "]/td")[4].text
  CAE=root.xpath("////*/div[@id = 'simuladorCreditoHipotecario']/table/tbody/tr["+ str(i) + "]/td")[5].text
  Gnotarial=root.xpath("////*/div[contains(@id,'targetSimulacion')]/div[1]/ul[1]/li[contains(@class,'list-group-item')]")[1].text 


Banco del Estado de Chile
Banco Bice
Scotiabank Chile
Coopeuch
Itaú Corpbanca
Banco de Crédito e Inversiones
Banco Santander-Chile
Banco de Chile
Banco Internacional
Banco Falabella


In [70]:
root.xpath("////*/div[contains(@id,'targetSimulacion')]/div[1]/ul[1]/li[contains(@class,'list-group-item')]")[1].text 

'\tGastos notariales: UF 2,50\xa0\n'

In [ ]:
cxv